In [1]:
!pip install langchain_openai langchain_community langchain pymysql chromadb -q langchain_community

In [1]:
db_user = "xxx"
db_password = "xxx"
db_host = "xxx"
db_name = "xxx"
from langchain_community.utilities.sql_database import SQLDatabase
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")
print(db.dialect)
print(db.get_usable_table_names())
print(db.table_info)

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on 'xxx' ([Errno 8] nodename nor servname provided, or not known)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [26]:
import os
os.environ['OPENAI_API_KEY']= "your api key"

In [27]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_community.agent_toolkits import create_sql_agent
llm = ChatOpenAI(model="gpt-4")
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)
# groq_api_key = ""
# llm = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="mixtral-8x7b-32768")

In [30]:
examples = [
    {
        "input": "請給我中信金2022年12月的營業費用",
        "query": """SELECT 營業費用 
                    FROM 金控損益表 
                    WHERE 代號 = '2891' AND 年_月 = '22-Dec'
                """
    },
    {
        "input": "我需要元大金2021年的現金流量總額.",
        "query": """SELECT SUM(本期產生現金流量) AS 元大金2021年現金流量總額
                    FROM 金控現金流量表
                    WHERE 代號 = '2885' AND 年_月 LIKE '21%'
                """
    },
    {
        "input": "請給我國票金2023和2022的資產總額和負債總額比較",
        "query": """SELECT 
                        SUM(CASE WHEN 年_月 LIKE '%23-Dec' THEN 資產總額 ELSE 0 END) AS 國泰金2023年資產總額,
                        SUM(CASE WHEN 年_月 LIKE '%22-Dec' THEN 資產總額 ELSE 0 END) AS 國泰金2022年資產總額,
                        SUM(CASE WHEN 年_月 LIKE '%23-Dec' THEN 負債總額 ELSE 0 END) AS 國泰金2023年負債總額,
                        SUM(CASE WHEN 年_月 LIKE '%22-Dec' THEN 負債總額 ELSE 0 END) AS 國泰金2022年負債總額
                    FROM 金控資產負債表
                    WHERE 代號 = '2889';
                """
    },
    {
        "input": "提供華南金2022年最高和最低的現金流量",
        "query": """SELECT MAX(本期產生現金流量) AS 華南金2022年最高現金流量, MIN(本期產生現金流量) AS 華南金2022年最低現金流量
                    FROM 金控現金流量表 
                    WHERE 代號 = '2880' AND 年_月 LIKE '22-%';
                """
    },
    {
        "input": "幫我比較富邦金和國泰金2023年12月的總損益",
        "query": """SELECT 名稱, 合併總損益
                    FROM 金控損益表
                    WHERE 代號 IN ('2881', '2882') AND 年_月 = '23-Dec';
                """
    },
    {
        "input": "查找永豐金2022年的最高每股盈餘月份",
        "query": """SELECT 年_月, 每股盈餘
                    FROM 金控損益表
                    WHERE 代號 = '2890' AND 年_月 LIKE '22-%'
                    ORDER BY 每股盈餘 DESC
                    LIMIT 1;
                """
    },
    {
        "input": "你好，幫我找出2022年營收成長率最高的一家金控",
        "query": """SELECT 代號, 名稱, `營收成長率(A)` AS 營收成長率
                    FROM 金控指標
                    WHERE 年_月 = '22-Dec'
                    ORDER BY 營收成長率 DESC
                    LIMIT 1;
                """
    },
    {
        "input": "Hi, 你可以給我2020和2019兩年總資產成長率前三高的金控嗎？",
        "query": """(SELECT 代號, 名稱, 總資產成長率, 年_月
                    FROM 金控指標
                    WHERE 年_月 = '20-Dec'
                    ORDER BY 總資產成長率 DESC
                    LIMIT 3)
                    UNION ALL
                    (SELECT 代號, 名稱, 總資產成長率, 年_月
                    FROM 金控指標
                    WHERE 年_月 = '19-Dec'
                    ORDER BY 總資產成長率 DESC
                    LIMIT 3);
                """
    }

 ]


In [31]:
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder,FewShotChatMessagePromptTemplate,PromptTemplate, FewShotPromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder

vectorstore = Chroma()
vectorstore.delete_collection()
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    vectorstore,
    k=3,
    input_keys=["input"],
)

example_prompt = PromptTemplate.from_template(
    "User input: {input}\nSQL query: {query}"
)

system_prefix = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct MySQL query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.

Here are some examples of user inputs and their corresponding SQL queries:"""


few_shot_prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    input_variables=["input","top_k"],
    prefix=system_prefix,
    suffix="" 
)
# print(few_shot_prompt.format(input="幫我找出2023年營收成長率最高的一家金控", top_k=5))


In [ ]:
full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=few_shot_prompt),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [ ]:
# Example formatted prompt
prompt_val = full_prompt.invoke(
    {
        "input": "幫我找出2023年營收成長率最高的一家金控",
        "top_k": 5,
        "agent_scratchpad": [],
    }
)
print(prompt_val.to_string())

In [ ]:
agent = create_sql_agent(
    llm=llm,
    db=db,
    prompt=full_prompt,
    verbose=True,
    agent_type="openai-tools",
)

In [ ]:
agent.invoke({"input": "給我中信金2023年12月的每股盈餘和資產總額"})